# Procesamiento de datos de potenciales evocados
Sobre el estudio de Bemis & Pylkkanen, 2011:  "Simple composition: A magnetoencephalography investigation into the comprehension of minimal linguistic phrases." The Journal of Neuroscience 31.8 (2011): 2801-2814.
Datos: Emilia Fló.

@author: Camila Zugarramurdi, Noviembre 2016


In [ ]:
# algunos settings
%config InlineBackend.figure_format = 'retina'
%pylab inline

In [ ]:
import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import scipy.io
import scipy.signal

from scipy import stats

Primero, levanto los datos crudos, corresponden a el EEG de 1 participante:

In [ ]:
filename = 'datoscrudos.mat' # Carga de datos
dataC = sp.io.loadmat(filename)


print (dataC.keys())

In [ ]:
# estas son todas las variables que había en los datos, puedo acceder a cada una con la sintaxis
dataC['numChan']

**¿Cuántos electrodos se usaron?**

In [ ]:
# la frecuencia de muestreo está en la key fs (por sampling frequency)
srate=dataC['fs'][0][0]
print (srate)
# los datos de la señal EEG están en la key data y los nombres de los electrodos en labels
dataRaw=dataC['data']
channels=dataC['labels']
# la última fila de los datos de EEG es la línea de triggers
triggers=dataRaw[-1,:]


In [ ]:
plt.plot(triggers)

**Triggers**: son los indicadores de eventos, definen cada ensayo. Cuando presento un estímulo importante, mando un trigger, así sé en qué punto de tiempo tengo que mirar en el registro de EEG para ver los potenciales evocados.

In [ ]:
print(triggers)

In [ ]:
plot(triggers)

In [ ]:
# Tengo que hacer algunas volteretas para obtener los puntos te tiempo de los triggers, 
# por ahora no importa, sí importa verlos:

moda=stats.mode(triggers)[0][0]

triggers=triggers-moda

#encuentro el tiempo del primer trigger de cada set
#**¿QUÉ QUIERE DECIR CADA SET'?**
nz=np.nonzero(triggers)[0]
loc=np.diff(nz)>1
index=nz[np.where(loc)[0]] #busco las ubicaciones donde el array lock es verdader
trig=[index,triggers[index]]



plt.figure(figsize=(10,3))
plt.plot(trig[0],trig[1],'ro')
plt.axis([75000,len(triggers),0,2.5])

print ('Hay '+ str(len(trig[0]))+ ' ensayos')

### En la figura anterior veo los triggers, cada uno define un ensayo. Hay aprox mitad en condición 1 y mitad en condición 2.

Bien, quiero ver varios canales al mismo tiempo, para ver cómo está la señal. Cada fila corresponde a un canal.


In [ ]:
# Podemos ignorar la definción de la función.

from matplotlib.collections import LineCollection

def plot_eeg(EEG, color='y',canales=1, tiempo=4):
    '''
    Grafico los datos de EEG, cada línea es un canal
    
    Parámetros
    ----------
    EEG : datos (canales x muestras)
    canales : el número de canales que quiero ver
    tiempo : el total de tiempo que quiero ver, en minutos
    color : caracter (default 'k', que es negro), otras opciones son 'r','g','b','k'

    '''
    vspace=0
    t=int(tiempo*srate*60)
    EEG=EEG[0:canales:,1:t] # me quedo con una parte, 15 canales 4 segundos
    bases = vspace * np.arange(len(EEG)) # vspace * 0, vspace * 1, vspace * 2, ..., vspace * 6
    
    # To add the bases (a vector of length 7) to the EEG (a 2-D Matrix), we don't use
    # loops, but rely on a NumPy feature called broadcasting:
    # http://docs.scipy.org/doc/numpy/user/basics.broadcasting.html
    EEG = EEG.T + bases
    
    # Calculate a timeline in seconds, knowing that the sample rate of the EEG recorder was 2048 Hz.
    samplerate = 256.
    time = np.arange(EEG.shape[0]) / samplerate
    
    # Plot EEG versus time
    plt.plot(time, EEG, color=color)
    
    # Add gridlines to the plot
    plt.grid()
    
    # Label the axes
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Canales')
  
    plt.gca().yaxis.set_ticks(bases)
   
    # Put a nice title on top of the plot
    plt.title('EEG data')


Llamando a la función plot_eeg con los parámetros tiempo (en minutos) y canales puedo ver los canales registrados. **¿Qué pasa con la señal a medida que avanza el tiempo?** Modificar el valor de tiempo y de canales en el código a continuación:

In [ ]:
plt.figure(figsize=(15,3))
plot_eeg(dataRaw, tiempo=1, canales=28, color = 'r')


Uno de los principales pasos en el procesamiento de datos de EEG para potenciales evocados es dividir la señal continua en cada uno de los ensayos. Los **ensayos** se definen desde una ventana de tiempo de X ms previa a la aparición del estímulo de interés hasta una ventana de tiempo posterior a la aparición. Típicamente estos valores corresponden a 200 ms previos y 800 ms posteriores. Además, debemos identificar, en cada ensayo, a qué condición corresponde. Lo hacemos a continuación:

Veamos cómo luce un ensayo:

In [ ]:
# Separo los triggers para cada condición
event_inds1 = trig[0][trig[1]==1]
event_inds2 = trig[0][trig[1]==2]

# Grafico un ensayo
this_event = event_inds1[1]
timewin = [int(srate*0.2), int(srate*0.8)] 
canal=8

my_ERP = dataRaw[canal][this_event-timewin[0]:this_event + timewin[1]]
my_ERP

plt.plot(my_ERP,label = 'ERP')
plt.title('esto es un ensayo')



Bien, pero el eje de las y está un poco raro, ¿qué unidades son esas? La señal está montada sobre una valor de voltaje de base. Necesitamos entonces hacer **correción por línea de base**: a cada ensayo le restamos el promedio del voltaje de una ventana de tiempo de aprox. 200 ms previos al comienzo del ensayo. De este modo todos los ensayos quedan centrados en 0, y podemos promediarlos.

Veamos si funcionó la corrección por línea de base:

In [ ]:
# Definición de ensayos y corrección por línea de base

# vamos a procesar solamente un canal para visualizarlo más fácilmente, en este caso 'Pz'
#probar para otros canales
dataRaw2 = dataRaw[8]
print ('el canal que vamos a procesar es '+ channels[8][0][0])


# en esta matriz guardo los datos de los ensayos: tiene tantas filas como ensayos (119) y tantas columnas como puntos de tiempo (255)

chan_baselines1 = np.zeros(len(event_inds1)); 
ERP_matrix1 = np.zeros((len(event_inds1),timewin[0]+timewin[1])); 

# Para cada ensayo de condición 1
for ensayo in range(len(event_inds1)):
    # 1.  calculo el promedio del voltaje antes de que empiece el ensayo 
    chan_baselines1[ensayo] = np.mean(dataRaw2[event_inds1[ensayo]-50:event_inds1[ensayo] + 0]) 
    # 2. y se lo resto a todo el ensayo
    ERP_baseline_removed = dataRaw2[:][event_inds1[ensayo]-timewin[0]:event_inds1[ensayo] + timewin[1]] - chan_baselines1[ensayo]     
    # 3. y voy guardando cada ensayo con la línea de base corregida en la matriz ERP_matrix
    ERP_matrix1[ensayo,:] = ERP_baseline_removed;
print ('la matriz de ensayos de la condición 1 tiene ' + str(ERP_matrix1.shape[0]) + ' filas')

# Para ensayos de la condición 2
chan_baselines2 = np.zeros(len(event_inds2));
ERP_matrix2 = np.zeros((len(event_inds2),timewin[0]+timewin[1])); 
# Para cada ensayo de condición 2,
for ensayo in range(len(event_inds2)):
    chan_baselines2[ensayo] = np.mean(dataRaw2[event_inds2[ensayo]-50:event_inds2[ensayo] + 0]) 
    ERP_baseline_removed = dataRaw2[:][event_inds2[ensayo]-timewin[0]:event_inds2[ensayo] + timewin[1]] - chan_baselines2[ensayo] 
    ERP_matrix2[ensayo,:] = ERP_baseline_removed;
print ('la matriz de ensayos de la condición 2 tiene ' + str(ERP_matrix2.shape[0])+  ' filas')


*¿Cuántos ensayos hay para cada condición?*

In [ ]:
# Grafico el primer ensayo:

plt.plot(ERP_matrix1[0],label = 'ERP')

Bien, ya está centrado en 0, pero el tiempo está medio raro, vamos a corregirlo también. Tengo 256 puntos de tiempo por segundo, y por lo tanto el eje x del gráfico de arriba no es tiempo en segundos, sino **puntos de tiempo o muestras**. ¿Cómo transformo tiempo a puntos de tiempo?


In [ ]:
# es una regla de 3, si 256 puntos de tiempo corresponden a 1 segundo, cuántos puntos de tiempo corresponden a X segundos.
# O sea, divido los puntos de tiempo por 256 [y le resto -0.2 porque quiero que el 0 corresponda al tiempo donde apareció el sustantivo]
tiempototal = ERP_matrix1.shape[1]
time = np.arange(tiempototal) / 256.-0.2

Pruebo volver a graficar, ahora indicando el tiempo en el x:

In [ ]:
ensayo=0
plt.plot(time, ERP_matrix1[ensayo],label = 'ERP: ensayo ' + str(ensayo+1), color='m')
plt.xlabel('Tiempo (s)')
plt.ylabel('Voltaje (uV)')
legend()


Bien, esto es 1 ensayo, pero tengo 120, aprox. la mitad por cada condición. Vamos a ver cómo está el resto. Ir aumentándo el número de ensayos a mostrar en la variable "trials" a continuación y graficar. *¿Hay diferencias entre las condiciones?*


In [ ]:
# el máximo número de ensayos es 120, la condicion es 1 o 2
trials = 45
condicion = 1

In [ ]:
# Grafico los ensayos

if condicion == 1: dat = ERP_matrix1
elif condicion == 2: dat = ERP_matrix2

plt.figure()
time = np.arange(dat.shape[1]) / 256.-0.2
for event in range(trials):
    plot(time, dat[event],label = 'ERP ensayo' +str(event+1))
if trials <= 5:
    plt.legend()

### ¿qué pasa cuando promedio los ensayos?


In [ ]:
mean_ERP1 = np.mean(ERP_matrix1,0) 
mean_ERP2 = np.mean(ERP_matrix2,0)

In [ ]:
time = np.arange(mean_ERP1.shape[0]) / 256.-0.2
plt.plot(time, mean_ERP1, label = 'ERP cond 1', color = 'g');
plt.plot(time, mean_ERP2, label = 'ERP cond 2', color = 'b');
legend()

### Vamos bien, pero ese ruido de alta frecuencia.... ¿se ve? Vamos a filtrar la señal. 
**Atención**: los filtros deben aplicarse antes de definir los ensayos, sobre la señal continua, hoy vamos a hacer una excepción. 

In [ ]:
#Defino las funciones de filtrado, es un filtro butterworth (especial para ingenieros)

# tomado prestado de acá: 
# http://localhost:8888/notebooks/Dropbox/tutorials-master/tutorials-master/Filters.ipynb

def butter_bandpass(lowcut, highcut, fs, order=4):
    #lowcut es el límite inferior del filtro
    #hicut es el límite superior del filtro
    #fs es la frecuencia de muestreo
    nyq = 0.5 * fs #nyquist frequency - see http://www.dspguide.com/ por más información
    low = float(lowcut) / nyq
    high = float(highcut) / nyq
    b, a = sp.signal.butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(mydata, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sp.signal.filtfilt(b, a, mydata)
    return y

def filter_dat(lo_cut,hi_cut,order):    

    filtdat = butter_bandpass_filter(dat,lo_cut,hi_cut,srate,order);
    fig = plt.figure(figsize=(24,6))
    ax1 = fig.add_subplot(2, 1, 1)
    plt.plot(dat,lw=2,color='blue',label = 'raw data')
    plt.plot(filtdat,lw=2,color='black',label='filtered data')
    plt.xlim(0, 1024)
    legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()
    return

In [ ]:
dataFilt1 = butter_bandpass_filter(mean_ERP1,0.5,40,256);
dataFilt2 = butter_bandpass_filter(mean_ERP2,0.5,40,256);

plt.plot(time,dataFilt1, label = 'ERP cond 1', color = 'g')
plt.plot(time,dataFilt2, label = 'ERP cond 2', color = 'b')
plt.legend()
plt.gca().invert_yaxis()

Esto se parece bastante a lo que vemos en los artículos, aunque normalmente está aún un poco más filtrado. En la literatura de lenguaje sin embargo, hay un fenómeno raro: se acostumbra a poner el negativo para arriba. En el comando anterior, descomentar la última línea para dar vuelta el eje de las y. *¿Podés agregar nombres en los ejes (tiempo y voltaje) y un título al gráfico?*

### Si el experimento funcionó, ¿a qué condición experimental (una palabra o dos palabras) crees que corresponde la condición 1 y a cuál la condición 2? 

En la vida real, seguimos estos mismo pasos para cada participante y para cada canal, y luego promediamos los promedios de cada participante y de cada canal, a esto le llamamos gran promedio. Cabe destacar que en este tutorial nos salteamos un paso fundamental que es limpiar los datos: los exploramos visualmente o con procedimientos un poquito más sofisticados (como con umbrales, la varianza por ensayo, o análisis de componentes principales) para eliminar ensayos contaminados por artefactos oculares, motores o de piel. Finalmente realizamos estadística inferencial para testear diferencias entre condiciones y/o definir ventanas de tiempo de interés.